In [1]:
# 매 강의 시작시마다 동일하게 처리해야 함
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:86% !important;}
div.cell.code_cell.rendered{width:100%;}
div.CodeMirror {font-family:Consolas; font-size:13pt;
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:10pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:10pt;padding:5px;}
table.dataframe{font-size:15px;
</style>
"""))

<b><font size="5" color="red">ch11. 데이터프레임과 시리즈</font></b>

# 9절 데이터 그룹화 및 집계

In [3]:
from statsmodels.api import datasets
iris = datasets.get_rdataset('iris').data
iris.iloc[::25]

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.1,3.5,1.4,0.2,setosa
25,5.0,3.0,1.6,0.2,setosa
50,7.0,3.2,4.7,1.4,versicolor
75,6.6,3.0,4.4,1.4,versicolor
100,6.3,3.3,6.0,2.5,virginica
125,7.2,3.2,6.0,1.8,virginica


## 9.1 Groupby
cf.pivot_table

In [5]:
# 1) 그룹화 방법 1.
#      df.groupby(by=[그룹핑할 열]). 기초통계함수()[함수적용해서 출력할 열들].기초통계함수()
#                [함수적용해서 출력할 열들] 생략시 그룹핑할 열을 제외한 모든 열 적용
iris_grouped = iris.groupby(iris.Species)
iris_grouped = iris.groupby(['Species'])
iris_grouped = iris.groupby('Species')
for group, data in iris_grouped:
    print(group)
    display(data.sample(2).sort_index())

setosa


,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
31,5.4,3.4,1.5,0.4,setosa
49,5.0,3.3,1.4,0.2,setosa


versicolor


,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
82,5.8,2.7,3.9,1.2,versicolor
89,5.5,2.5,4.0,1.3,versicolor


virginica


,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
123,6.3,2.7,4.9,1.8,virginica
148,6.2,3.4,5.4,2.3,virginica


In [6]:
# 잘 모르겠다~ 싶으면 요 부분만 잘 알고 있어도 됨

iris.groupby(by='Species').mean()[['Petal.Length','Petal.Width']]
iris.groupby(by='Species')[['Petal.Length','Petal.Width']].mean()

,Petal.Length,Petal.Width
Species,,
setosa,1.462,0.246
versicolor,4.260,1.326
virginica,5.552,2.026


In [7]:
iris_grouped_sum = iris.groupby('Species').median()
# iris_grouped_sum.reset_index()
iris_grouped_sum

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
Species,,,,
setosa,5.0,3.4,1.50,0.2
versicolor,5.9,2.8,4.35,1.3
virginica,6.5,3.0,5.55,2.0


In [8]:
iris_grouped_sum.loc[['setosa','versicolor']]


,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
Species,,,,
setosa,5.0,3.4,1.50,0.2
versicolor,5.9,2.8,4.35,1.3


In [9]:

iris_grouped_sum2 = iris.groupby('Species', as_index=False).sum()
iris_grouped_sum2

,Species,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
0,setosa,250.3,171.4,73.1,12.3
1,versicolor,296.8,138.5,213.0,66.3
2,virginica,329.4,148.7,277.6,101.3


In [10]:
iris_grouped_sum2[(iris_grouped_sum2.Species=='setosa') | 
                  (iris_grouped_sum2.Species=='versicolor')]

,Species,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
0,setosa,250.3,171.4,73.1,12.3
1,versicolor,296.8,138.5,213.0,66.3


In [11]:
#     df.groupby(by=[그룹핑할 열])[함수적용해서 출력할 열들].함수()
# 2) 그룹화 방법 2
#     df.pivot_table(index=[그룹핑할열],
#                    values=[함수적용해서 출력할 일들],
#                    aggfunc=함수(기본값:mean))

In [12]:

iris.groupby('Species')['Petal.Length', 'Petal.Width'].mean()
iris.pivot_table(index='Species',
                values=['Petal.Length', 'Petal.Width'],
                aggfunc='mean')

ValueError: Cannot subset columns with a tuple with more than one element. Use a list instead.

In [28]:
iris.groupby('Species').count() # 결측치를 제외한 값의 갯수
iris.pivot_table(index='Species',
                #values=['Sepal.Length','Sepal.Width','Petal.Length','Petal.Width'],
                 aggfunc='count')

,Petal.Length,Petal.Width,Sepal.Length,Sepal.Width
Species,,,,
setosa,50,50,50,50
versicolor,50,50,50,50
virginica,50,50,50,50


In [ ]:
iris.head()

## 다중열로 구룹화

In [30]:
# iris로 다중열로 그룹화하기 위해 데이터 컬럼 추가(num 필드)
iris.shape

(150, 5)

In [32]:
import numpy as np
np.ravel([[i]*25 for i in range(6)])

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5])

In [34]:
iris['num'] = np.ravel([[i]*25 for i in range(6)])
iris.iloc[::25]

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species,num
0,5.1,3.5,1.4,0.2,setosa,0
25,5.0,3.0,1.6,0.2,setosa,1
50,7.0,3.2,4.7,1.4,versicolor,2
75,6.6,3.0,4.4,1.4,versicolor,3
100,6.3,3.3,6.0,2.5,virginica,4
125,7.2,3.2,6.0,1.8,virginica,5


In [36]:
# 각열의 임의의 데이터에 결측치를 할당
# iris.iloc[0, 0] = np.nan

In [38]:
import random
print(random.sample(range(149), 3))
iris.iloc[random.sample(range(149), 3), 0]

[23, 14, 136]


48    5.3
34    4.9
81    5.5
Name: Sepal.Length, dtype: float64

In [40]:
iris.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Sepal.Length  150 non-null    float64
 1   Sepal.Width   150 non-null    float64
 2   Petal.Length  150 non-null    float64
 3   Petal.Width   150 non-null    float64
 4   Species       150 non-null    object 
 5   num           150 non-null    int64  
dtypes: float64(4), int64(1), object(1)
memory usage: 7.2+ KB


In [42]:
iris.groupby(by=['Species','num'], 
            # as_index=False
            )[['Petal.Length','Petal.Width']].count()

Petal.Length  Petal.Width
Species    num                           
setosa     0              25           25
           1              25           25
versicolor 2              25           25
           3              25           25
virginica  4              25           25
           5              25           25

In [44]:

iris.pivot_table(index=['Species', 'num'],
                values = ['Petal.Length', 'Petal.Width'],
                aggfunc='count')

Petal.Length  Petal.Width
Species    num                           
setosa     0              25           25
           1              25           25
versicolor 2              25           25
           3              25           25
virginica  4              25           25
           5              25           25

In [46]:
iris.groupby('Species').describe().T

Species                setosa  versicolor  virginica
Sepal.Length count  50.000000   50.000000  50.000000
             mean    5.006000    5.936000   6.588000
             std     0.352490    0.516171   0.635880
             min     4.300000    4.900000   4.900000
             25%     4.800000    5.600000   6.225000
             50%     5.000000    5.900000   6.500000
             75%     5.200000    6.300000   6.900000
             max     5.800000    7.000000   7.900000
Sepal.Width  count  50.000000   50.000000  50.000000
             mean    3.428000    2.770000   2.974000
             std     0.379064    0.313798   0.322497
             min     2.300000    2.000000   2.200000
             25%     3.200000    2.525000   2.800000
             50%     3.400000    2.800000   3.000000
             75%     3.675000    3.000000   3.175000
             max     4.400000    3.400000   3.800000
Petal.Length count  50.000000   50.000000  50.000000
             mean    1.462000    4.260000   5.552000
             std     0.173664    0.469911   0.551895
             min     1.000000    3.000000   4.500000
             25%     1.400000    4.000000   5.100000
             50%     1.500000    4.350000   5.550000
             75%     1.575000    4.600000   5.875000
             max     1.900000    5.100000   6.900000
Petal.Width  count  50.000000   50.000000  50.000000
             mean    0.246000    1.326000   2.026000
             std     0.105386    0.197753   0.274650
             min     0.100000    1.000000   1.400000
             25%     0.200000    1.200000   1.800000
             50%     0.200000    1.300000   2.000000
             75%     0.300000    1.500000   2.300000
             max     0.600000    1.800000   2.500000
num          count  50.000000   50.000000  50.000000
             mean    0.500000    2.500000   4.500000
             std     0.505076    0.505076   0.505076
             min     0.000000    2.000000   4.000000
             25%     0.000000    2.000000   4.000000
             50%     0.500000    2.500000   4.500000
             75%     1.000000    3.000000   5.000000
             max     1.000000    3.000000   5.000000

## 9.2 레이블 인코딩

In [49]:
iris.drop('num', axis=1, inplace=True)

In [51]:
# 레이블을 인코딩을 지원하는 클래스 import
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder() # 객체
iris['target'] = le.fit_transform(iris.Species)
iris.iloc[::25]

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species,target
0,5.1,3.5,1.4,0.2,setosa,0
25,5.0,3.0,1.6,0.2,setosa,0
50,7.0,3.2,4.7,1.4,versicolor,1
75,6.6,3.0,4.4,1.4,versicolor,1
100,6.3,3.3,6.0,2.5,virginica,2
125,7.2,3.2,6.0,1.8,virginica,2


# 10절. 데이터 구조 변경
- 와이드 포맷 <-> 롱포맷
- 판다스 공식 문서의 melt : https://pandas.pydata.org/docs/user_guide/reshaping.html#reshaping-by-melt
- Tidy Data(7p) : https://vita.had.co.nz/papers/tidy-data.pdf  

In [54]:
from statsmodels.api import datasets
airquality = datasets.get_rdataset('airquality',
                                   package='datasets').data
airquality.head(2)

,Ozone,Solar.R,Wind,Temp,Month,Day
0,41.0,190.0,7.4,67,5,1
1,36.0,118.0,8.0,72,5,2


## 10.1 melt를 이용한 언피벗팅(와이드포맷 -> 롱포맷)

In [57]:
air_melted =  airquality.melt(id_vars=['Month','Day'],
               var_name='변수명', # 기본값:variable
               value_name='값') # 기본값 : value
air_melted.iloc[::20]
air_melted.sort_values(by=['Month','Day'], inplace=True)
air_melted

,Month,Day,변수명,값
0,5,1,Ozone,41.0
153,5,1,Solar.R,190.0
306,5,1,Wind,7.4
459,5,1,Temp,67.0
1,5,2,Ozone,36.0
...,...,...,...,...
610,9,29,Temp,76.0
152,9,30,Ozone,20.0
305,9,30,Solar.R,223.0
458,9,30,Wind,11.5


In [59]:
# melt 연습하기 (부동산 와이드포맷 데이터를 롱포맷)
%ls c:\ai_x\download\shareData\부동산_250213\

ls: c:ai_xdownloadshareData부동산_250213: No such file or directory


In [62]:
import pandas as pd
df = pd.read_csv(r'c:\ai_x\download\shareData\부동산_250213\전국 평균 평당 분양가격(2013년 9월부터 2015년 8월까지).csv', encoding='cp949')
ㅐ


FileNotFoundError: [Errno 2] No such file or directory: 'c:\\ai_x\\download\\shareData\\부동산_250213\\전국 평균 평당 분양가격(2013년 9월부터 2015년 8월까지).csv'

In [64]:

new_df = df.melt(id_vars='지역',
           var_name='연도월',
           value_name='평당분양가')
new_df.head()

NameError: name 'df' is not defined

## 10.2 pivot_table을 이용한 피벗팅(롱포맷->와이드포맷) 

In [ ]:
# air_meltied를 와이드 포맷으로
air_melted.head()

In [66]:
airquality2 = air_melted.pivot_table(index=['Month','Day'],
                      columns='변수명',
                      values='값')
airquality2.reset_index(inplace=True)
airquality2.columns.name = None


In [ ]:
airquality2.head()

# 11절. 데이터프레임(시리즈)에 함수 적용시키기
- apply(func, axis) : 축요소별(요소별) 함수 적용(데이터 프레임이나 시리즈에서 가능)
- applymap(func) : 각 요소별ㄹ 함수 적용(데이터 프레임에서만 가능)
- map(func) : 각 요소별 함수 적용(1차원 리스트, 시리즈에서만 가능)

In [68]:
new_df.head()

NameError: name 'new_df' is not defined

## 11.1 apply

In [70]:
member = pd.read_csv('data/ch11_member.csv', comment='#')
member

,Name,Age,Email,Address,Birth
0,홍길동,20,hong@hong.com,서울시 강동구,1999-01-01
1,홍길서,30,seo@hong.com,서울시 중구,1999-12-01
2,홍길남,21,nam@hong.com,서울시 강동구,1999-11-01
3,홍길북,22,buk@hong.com,서울시 관악구,1999-05-01
4,홍길중,23,mid@hong.com,부산시 중구,1999-01-30


In [72]:
def get_year(datestr):
    return int(datestr.split('-')[0])

In [74]:
get_year = lambda datestr : int(datestr.split('-')[0])

In [76]:
get_year('1999-05-01')

1999

In [78]:
get_month = lambda datestr : int(datestr.split('-')[1])

In [80]:
get_month('1999-05-01')

5

In [82]:
member['year'] = member['Birth'].apply(lambda datestr : int(datestr.split('-')[0]))
member.head()

,Name,Age,Email,Address,Birth,year
0,홍길동,20,hong@hong.com,서울시 강동구,1999-01-01,1999
1,홍길서,30,seo@hong.com,서울시 중구,1999-12-01,1999
2,홍길남,21,nam@hong.com,서울시 강동구,1999-11-01,1999
3,홍길북,22,buk@hong.com,서울시 관악구,1999-05-01,1999
4,홍길중,23,mid@hong.com,부산시 중구,1999-01-30,1999


In [84]:
X = iris.iloc[:, :-2]
# X의 열별 평균
avg = X.mean(axis=0)
avg

Sepal.Length    5.843333
Sepal.Width     3.057333
Petal.Length    3.758000
Petal.Width     1.199333
dtype: float64

In [86]:
X.iloc[0] -avg

Sepal.Length   -0.743333
Sepal.Width     0.442667
Petal.Length   -2.358000
Petal.Width    -0.999333
dtype: float64

In [ ]:
# X의 모든행에 대하여 열별 평균과의 거리를 소수점 2자리까지 출력
X.apply(lambda x : round(x-avg, 2), axis=1)
# X.apply(lambda x: (x - avg).round(2), axis=1)

## 11.2 applymap

In [88]:
X = iris.iloc[:, :-2]

In [90]:
X.applymap(np.round)

/var/folders/6s/4rdt6bws0gq70r6hvc61y5cr0000gn/T/ipykernel_3502/974435539.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  X.applymap(np.round)


,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
0,5.0,4.0,1.0,0.0
1,5.0,3.0,1.0,0.0
2,5.0,3.0,1.0,0.0
3,5.0,3.0,2.0,0.0
4,5.0,4.0,1.0,0.0
...,...,...,...,...
145,7.0,3.0,5.0,2.0
146,6.0,2.0,5.0,2.0
147,6.0,3.0,5.0,2.0
148,6.0,3.0,5.0,2.0


## 11.3 map
- 1차원

In [92]:
member['month'] = member['Birth'].map(lambda datestr : int(datestr.split('-')[1]))
member

,Name,Age,Email,Address,Birth,year,month
0,홍길동,20,hong@hong.com,서울시 강동구,1999-01-01,1999,1
1,홍길서,30,seo@hong.com,서울시 중구,1999-12-01,1999,12
2,홍길남,21,nam@hong.com,서울시 강동구,1999-11-01,1999,11
3,홍길북,22,buk@hong.com,서울시 관악구,1999-05-01,1999,5
4,홍길중,23,mid@hong.com,부산시 중구,1999-01-30,1999,1


In [94]:
# 연습하기(appy, map)
# new_df의 연도월 데이터를 연도와월을 int로 분리하여 컬럼을 추가하세요
new_df.sample()

NameError: name 'new_df' is not defined

In [96]:
new_df['연도'] = new_df['연도월'].apply(lambda str:int(str.split('년')[0]))
new_df['월'] = new_df['연도월'].map(lambda str:int(str.split('년')[1][:-1]))
new_df.info()

NameError: name 'new_df' is not defined

In [98]:
new_df.sample()

NameError: name 'new_df' is not defined

In [100]:
new_df.drop(['연도','월'], axis=1, inplace=True)

NameError: name 'new_df' is not defined

In [102]:
# ★ 2차원 데이터 프레임의 apply 이용하는 방법

In [ ]:
new_df.head(2)

In [ ]:
def year_month_append(df):
    'df를 받아 연도와 월 추가후 reutrn'
    df = df.copy() # 깊은 복사
    df['연도'] = int(df['연도월'].split('년')[0])
    df['월'] = int(df['연도월'].split('년')[1].replace('월',''))
    return df

In [ ]:
year_month_append(new_df.iloc[0])

In [ ]:
new_df = new_df.apply(year_month_append, axis=1)
new_df.head()

# 12절. 일괄변경하기(결측치, 특정값)
## 12.1 fillna
- 대부분 결측치는 drop하거나, 평균값대체, 중위수대체, 그룹별 중위수, 이전값, 이후값 등으로 대체

In [105]:
df = pd.DataFrame([[np.nan, 2, np.nan, 0],
                   [3,      4, np.nan, 1],
                   [np.nan, 3, np.nan, 5],
                   [np.nan, 3, np.nan, 4]],
                 columns=[list('ABCD')])
df

,A,B,C,D
0,NaN,2,NaN,0
1,3.0,4,NaN,1
2,NaN,3,NaN,5
3,NaN,3,NaN,4


In [107]:
# 모든 결측치를 0 대체
df.fillna(0)

,A,B,C,D
0,0.0,2,0.0,0
1,3.0,4,0.0,1
2,0.0,3,0.0,5
3,0.0,3,0.0,4


In [109]:
# 결측치를 이전행의 값으로 대체
df.fillna(method='ffill')

/var/folders/6s/4rdt6bws0gq70r6hvc61y5cr0000gn/T/ipykernel_3502/4292330197.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill')


,A,B,C,D
0,NaN,2,NaN,0
1,3.0,4,NaN,1
2,3.0,3,NaN,5
3,3.0,3,NaN,4


In [111]:
# 결측치를 다음행의 값으로 대체
df.fillna(method='bfill')

/var/folders/6s/4rdt6bws0gq70r6hvc61y5cr0000gn/T/ipykernel_3502/4039330315.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='bfill')


,A,B,C,D
0,3.0,2,NaN,0
1,3.0,4,NaN,1
2,NaN,3,NaN,5
3,NaN,3,NaN,4


In [113]:
# 결측치를 열마다 특정값으로 대체
values = {'A':85, 'B':80, 'C':75, 'D':90}
df.fillna(value=values)

,A,B,C,D
0,85.0,2,75.0,0
1,3.0,4,75.0,1
2,85.0,3,75.0,5
3,85.0,3,75.0,4


In [115]:
# 결측치를 열별 평균값으로 대체
df.mean(axis=0)
df.fillna(value=df.mean(axis=0))

,A,B,C,D
0,3.0,2,NaN,0
1,3.0,4,NaN,1
2,3.0,3,NaN,5
3,3.0,3,NaN,4


In [117]:
# 결측치 대체를 2번째 결측치까지만 대체
print(values)
df.fillna(value=values, limit=2)

{'A': 85, 'B': 80, 'C': 75, 'D': 90}


,A,B,C,D
0,85.0,2,75.0,0
1,3.0,4,75.0,1
2,85.0,3,NaN,5
3,NaN,3,NaN,4


## 12.2 replace(to_value, new_value, inplace=T/F)
- to_value를 new_value
- to_value에 정규표현식 이용